In [1]:
import os
import lightgbm as lgb
import pandas as pd
import warnings
import numpy as np
import random
import ast


# importo los .py
import sys
sys.path.append("auxiliares")
import config
import metricas
import lightgbm_aux
import extras

pd.options.display.float_format = '{:.2f}'.format
pd.set_option('display.max_columns', None)

# Ignore all warnings
warnings.filterwarnings("ignore")

In [2]:
####################################################
############# Setear segun cada maquina ############
#os.chdir("C:/Users/herna/labo3_empresa3_repo/")
#os.chdir("C:/diego_tools/labo3/")
os.chdir("/home/dcastro_contacto/buckets/b1/")
####################################################

In [3]:
#Manual
prefijo_arch_entrada = "datasets/emp3_sellout_fe_cluster_manual_"
num_clusters=94
carpeta_base_exp = "exp/avanzada_1/"

#Time Warping
#prefijo_arch_entrada = "datasets/emp3_sellout_base_cluster_warp_"
#prefijo_arch_salida = "datasets/emp3_sellout_fe_cluster_warp_"
#num_clusters=94
#carpeta_exp_prefijo = "exp/avanzada_2/"

In [4]:
EXP_NOMBRE = "20231214_142500"

In [5]:
def lgbm_error_rate(preds, train_data):
    labels = train_data.get_label()
    ret_val = metricas.error_rate(labels,preds)
    return 'ER', ret_val, False

In [6]:
##############################################

In [7]:
carpeta_exp = carpeta_base_exp + EXP_NOMBRE + "/"

In [8]:
df_prods_prediccion_future=pd.read_csv(config.ARCH_PRODUCTOS_PREDICCION_FUTURE)
df_prods_prediccion_future.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 780 entries, 0 to 779
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   product_id  780 non-null    int64
dtypes: int64(1)
memory usage: 6.2 KB


In [9]:
df_opt_bay = pd.read_excel(carpeta_exp + "opt_bay.xlsx")
df_opt_bay.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94 entries, 0 to 93
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   cluster         94 non-null     int64  
 1   error_validate  86 non-null     float64
 2   lgbm_params     86 non-null     object 
 3   best_iteration  86 non-null     float64
dtypes: float64(2), int64(1), object(1)
memory usage: 3.1+ KB


In [10]:
df_opt_bay.isna().sum()

cluster           0
error_validate    8
lgbm_params       8
best_iteration    8
dtype: int64

In [11]:
df_opt_bay = df_opt_bay.fillna("-1") 
df_opt_bay.head(50)

,cluster,error_validate,lgbm_params,best_iteration
0,0,87.78,"{'objective': 'regression', 'first_metric_only...",616.00
1,1,68.49,"{'objective': 'regression', 'first_metric_only...",53.00
2,2,107.66,"{'objective': 'regression', 'first_metric_only...",1103.00
3,3,86.78,"{'objective': 'regression', 'first_metric_only...",194.00
4,4,85.85,"{'objective': 'regression', 'first_metric_only...",53.00
5,5,119.71,"{'objective': 'regression', 'first_metric_only...",78.00
6,6,75.66,"{'objective': 'regression', 'first_metric_only...",482.00
7,7,67.65,"{'objective': 'regression', 'first_metric_only...",357.00
8,8,86.75,"{'objective': 'regression', 'first_metric_only...",307.00
9,9,66.56,"{'objective': 'regression', 'first_metric_only...",891.00


In [12]:
prods_iter = []
custs_iter = []
preds_iter = []

for i in range(0,num_clusters):
    print("Cluster", i)

    df_sellout = pd.read_csv(prefijo_arch_entrada + str(i) + ".csv")
        
    ### Categoricas
    df_sellout = extras.convertir_categoricas_prod_cust(df_sellout)

    ### Remover columnas
    df_sellout = df_sellout.drop(columns=["periodo_fecha","brand"])
    
    if df_opt_bay.iloc[i].lgbm_params!="-1":
        
        # Modelo Final
        cols_remover_entren = ["tn_mas_2","periodo"]
        
        df_train_modelo_final = df_sellout[(df_sellout.periodo <= config.MODELO_FINAL_PERIODO_LIMITE_PARAM) & (df_sellout.periodo >= config.PERIODO_INICIO_PARAM)]
        df_future = df_sellout[(df_sellout.periodo == config.FUTURE_PERIODO_PARAM)]
        
        print("Periodos entrenar MODELO FINAL:",df_train_modelo_final.periodo.unique())
        print("Periodos FUTURE:",df_future.periodo.unique())
        
        # Variables entrenamiento
        X_train_modelo_final = df_train_modelo_final.drop(columns=cols_remover_entren, axis=1)
        y_train_modelo_final = df_train_modelo_final.tn_mas_2
        cols_entren = X_train_modelo_final.columns.tolist()
        
        X_future = df_future.drop(columns=cols_remover_entren, axis=1)
        
        print("\nX_train_all:", X_train_modelo_final.shape)
        print("y_train_all:", y_train_modelo_final.shape)
        print("\nX_future:", X_future.shape)
        
        lgbtrain_modelo_final = lgb.Dataset(data=X_train_modelo_final, label=y_train_modelo_final, feature_name=cols_entren)
        
        future_params = ast.literal_eval(df_opt_bay.iloc[i].lgbm_params) #convierto string a diccionario
        future_best_iter = int(df_opt_bay.iloc[i].best_iteration)
        y_pred_future = lightgbm_aux.semillerio(lgbtrain_modelo_final, future_params, future_best_iter, X_future,config.CANT_SEMILLAS)
        
        prods_iter.extend(X_future.product_id)
        custs_iter.extend(X_future.customer_id)
        preds_iter.extend(y_pred_future)
    else:
        print("Sin datos en el cluster")

        # Completo con prediccion 0
        df_sellout = df_sellout[["product_id","customer_id"]].drop_duplicates()
        df_sellout["pred"] = 0
        prods_iter.extend(df_sellout.product_id)
        custs_iter.extend(df_sellout.customer_id)
        preds_iter.extend(df_sellout.pred)

Cluster 0
Periodos entrenar MODELO FINAL: [201701 201702 201703 201704 201705 201706 201707 201708 201709 201710
 201711 201712 201801 201802 201803 201804 201805 201806 201807 201808
 201809 201810 201811 201812 201901 201902 201903 201904 201905 201906
 201907 201909 201910]
Periodos FUTURE: [201912]

X_train_all: (798001, 120)
y_train_all: (798001,)

X_future: (34626, 120)
Cluster 1
Periodos entrenar MODELO FINAL: [201701 201702 201703 201704 201705 201706 201707 201708 201709 201710
 201711 201712 201801 201802 201803 201804 201805 201806 201807 201808
 201809 201810 201811 201812 201901 201902 201903 201904 201905 201906
 201907 201909 201910]
Periodos FUTURE: [201912]

X_train_all: (582501, 120)
y_train_all: (582501,)

X_future: (21492, 120)
Cluster 2
Periodos entrenar MODELO FINAL: [201701 201702 201703 201704 201705 201706 201707 201708 201709 201710
 201711 201712 201801 201802 201803 201804 201805 201806 201807 201808
 201809 201810 201811 201812 201901 201902 201903 201904 2

In [13]:
df_prediccion = pd.DataFrame(data={"product_id":prods_iter,"customer_id":custs_iter,"tn":preds_iter})

#Competencia 1
df_prediccion_prod = df_prediccion.groupby("product_id")["tn"].sum().reset_index()
df_prediccion_prod.tn =  extras.limpiar_prediccion(df_prediccion_prod.tn)
df_prediccion_prod = df_prediccion_prod[df_prediccion_prod.product_id.isin(df_prods_prediccion_future.product_id)]
df_prediccion_prod.to_csv(carpeta_exp + "prediccion_comp1.csv",index=False)
print(df_prediccion_prod.head())
print(len(df_prediccion_prod))

# Competencia 2
star_products = [20001, 20002, 20003, 20004, 20005, 20006, 20007, 20009, 20011, 20032]
star_customers = [10001, 10002, 10003, 10004, 10005, 10006, 10007, 10008, 10009, 10011, 10012, 10013]
df_prediccion["product_customer_id"]="P"+df_prediccion["product_id"].astype(str) + '-C' + df_prediccion["customer_id"].astype(str)
df_prediccion = df_prediccion[(df_prediccion.product_id.isin(star_products)) & (df_prediccion.customer_id.isin(star_customers))]
df_prediccion = df_prediccion.drop(columns=["product_id","customer_id"])
df_prediccion.to_csv(carpeta_exp + "prediccion_comp2.csv",index=False)
print(df_prediccion.head())
print(len(df_prediccion))

   product_id      tn
0       20001 1577.43
1       20002 1142.30
2       20003  725.84
3       20004  608.74
4       20005  583.13
780
      tn product_customer_id
0 153.22       P20001-C10001
1 114.15       P20001-C10002
2 144.11       P20001-C10003
3 141.63       P20001-C10004
4  30.16       P20001-C10005
120
